## Training: Learning Rates

The learning rate is one of the most important hyperparameters when training a neural network. In general, the optimal learning rate is about half of the maximum learning rate (i.e., the learning rate above which the training algorithm diverges - that is, where it's performance begins to suffer). One way to find a good learning rate is to train the model for a few hundred iterations, starting with a very low learning rate (e.g., $10^{–5}$) and gradually increasing it up to a very large value (e.g., 10). This is done by multiplying the learning rate by a constant factor at each iteration (e.g., to go from $10^{–5}$ to 10 in 500 iterations). 

If you plot the loss as a function of the learning rate (using a log scale for the learning rate), you should see it dropping at first. But after a while, the learning rate will be too large, so the loss will shoot back up: the optimal learning rate will be a bit lower than the point at which the loss starts to climb (typically about 10 times lower than the turning point). You can then reinitialize your model and train it normally using this good learning rate.  Here's how to do this in code.

In [1]:
# Using Fashion MNIST as a working example
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

def build_model(seed=42):
    tf.random.set_seed(seed)
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

def build_and_train_model(optimizer):
    model = build_model()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model.fit(X_train, y_train, epochs=10,
                     validation_data=(X_valid, y_valid))

**Step 1:  Increasing the learning rate**

The following callback increases the learning rate by a factor in each batch, and also keeps track of the losses.  This is a little kludgey because keras only reports mean loss (over all batches thus far) so we need to keep track of things to figure out how much loss is incurred by each batch. 

In [6]:
import math
import os
import numpy as np
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
K = tf.keras.backend
from keras import backend as K

class ExponentialLearningRate(keras.callbacks.Callback):
    def init(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []

    def on_epoch_begin(self, epoch, logs=None):
        self.sum_of_epoch_losses = 0

    def on_batch_end(self, batch, logs=None):
        mean_epoch_loss = logs["loss"]  # the epoch's mean loss so far 
        new_sum_of_epoch_losses = mean_epoch_loss * (batch + 1)
        batch_loss = new_sum_of_epoch_losses - self.sum_of_epoch_losses
        self.sum_of_epoch_losses = new_sum_of_epoch_losses
        self.rates.append(float(np.array(self.model.optimizer.learning_rate)))
        self.losses.append(batch_loss)

**Step 2: Probing the model**

The following routine trains a model using the above callback to update the learning rate through a range of values, and then resets the model when done.

In [3]:
def find_learning_rate(model, X, y, epochs=1, batch_size=32, min_rate=1e-4,
                       max_rate=1):
    init_weights = model.get_weights()
    iterations = math.ceil(len(X) / batch_size) * epochs
    factor = (max_rate / min_rate) ** (1 / iterations)
    init_lr = model.optimizer.learning_rate
    model.optimizer.learning_rate = min_rate
    exp_lr = ExponentialLearningRate(factor)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size,
                        callbacks=[exp_lr])
    model.optimizer.learning_rate = init_lr
    model.set_weights(init_weights)
    return exp_lr.rates, exp_lr.losses

**Step 3: Inspecting the data**

Finally, we simply plot the losses against the learning rate to identify a "peak" and then we'll back off from there.

In [4]:
import matplotlib.pyplot as plt

def plot_lr_vs_loss(rates, losses):
    plt.plot(rates, losses, "b")
    plt.gca().set_xscale('log')
    max_loss = losses[0] + min(losses)
    plt.hlines(min(losses), min(rates), max(rates), color="k")
    plt.axis([min(rates), max(rates), 0, max_loss])
    plt.xlabel("Learning rate")
    plt.ylabel("Loss")
    plt.grid()

Now we build the model and find the optimal learning rate.

In [5]:

optimal_lr = 0.1
optimizer = tf.keras.optimizers.SGD(learning_rate=optimal_lr)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,metrics=["accuracy"])
history_optimal_lr = model.fit(X_train, y_train, epochs=10,
                     validation_data=(X_valid, y_valid))

NameError: name 'model' is not defined

This suggests an optimal learning rate around $ 10^{-1} $.  So, let's give it a try.

In [ ]:
optimal_lr = 0.1
optimizer = tf.keras.optimizers.SGD(learning_rate=optimal_lr)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,metrics=["accuracy"])
history_optimal_lr = model.fit(X_train, y_train, epochs=10,
                     validation_data=(X_valid, y_valid))

### Exercise

Verify that 0.1 is a better learning rate than something much smaller (e.g. .001) or much larger (e.g., .5).

### Learning Rate Schedules

A constant learning rate doesn't account for the possibility that the gradient landscape changes as we move across it.  Optimizers can help address this, but it is also possible to adjust the learning rate according to a schedule.  In the following, we discuss a few approaches for this.

#### Motivation?

1. **Early Training:** Initially, a larger learning rate can be beneficial, enabling rapid learning and helping the model to quickly escape suboptimal local minima.
2. **Later Stages:** As training progresses, reducing the learning rate helps to stabilize the learning process and fine-tune the model parameters, leading to better convergence.
3. **Preventing Overfitting and Oscillations:** A carefully chosen learning rate schedule can prevent the model from overfitting and reduce oscillations near the minima.

<div style="text-align: center">

![learning rate](assets/learning_rate.png)
</div>

Learning rate scheduling is thus a balancing act - starting with a higher learning rate for faster convergence initially, then decreasing it to allow more fine-grained adjustments as the model starts converging.

### Methods

1. **Power Scheduling:**
   - **Description:** Gradually decreases the learning rate over time using a polynomial decay, typically quadratic.
   - **Formulation:** $ \eta(t) = \eta_0 / (1 + t/s)^c $, where $ \eta_0 $ is the initial learning rate, $ t $ is the iteration number, $ s $ is a step parameter, and $ c $ is the power, often set to 1.


In [ ]:
# Keras supports learning rate scheduling by extending the LearningRateSchedule class, and then passing that to a optimizer.  To implement a PowerSchedule, we can do the following:

class PowerScheduling(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_rate, step_param):
        super(PowerScheduling, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_rate = decay_rate
        self.step_param = step_param

    def __call__(self, step):
        # Note the "cast" here - necessary to make sure we get the correct type back
        return self.initial_learning_rate / (1 + tf.cast(step, tf.float32) / self.step_param) ** self.decay_rate


# Example usage
initial_learning_rate = 0.1
decay_rate = 1.0  # For linear decay
step_param = 1000.0  # Determines how quickly the learning rate decreases

lr_schedule = PowerScheduling(initial_learning_rate, decay_rate, step_param)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
history_power_scheduling = build_and_train_model(optimizer)


2. **Exponential Scheduling:**
   - **Description:** Cuts the learning rate by a factor of $ \gamma $ every $ s $ steps.
   - **Formulation:** $ \eta(t) = \eta_0 \cdot \gamma^{t/s} $.
   - **Senior et al's Recommendation:** Particularly recommended for its effectiveness and simplicity in many applications.


In [ ]:
import math

# First, we'll calculate a learning rate based on the number of batches we anticipate
n_epochs = 10
batch_size = 32
n_steps = n_epochs * math.ceil(len(X_train) / batch_size)

lr0=.1

# Keras now has an ExponentialDecay scheduling class we can use here
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr0,
                                                             decay_steps=n_steps,
                                                             decay_rate=.95)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
history_exponential_scheduling = build_and_train_model(optimizer)


3. **Piecewise Constant Scheduling:**
   - **Description:** Uses a constant learning rate for a set number of epochs and then lowers it to another constant rate for another set of epochs, and so on.
   - **Formulation:** Implementing a pre-defined series of constant learning rates at different training stages.


In [ ]:
# Let's say we want a learning rate of .1 for the first 3 epochs, .05 for the next three, and then .005 for the remainder.  So, we'll calculate the number of batches per epoch given our data.
epoch_steps = math.ceil(len(X_train) / batch_size)
b1 = epoch_steps * 3
b2 = b1+epoch_steps * 3


boundaries = [b1, b2]
values = [.1, 0.05, 0.005]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
history_piecewise = build_and_train_model(optimizer)


4. **Performance Scheduling:**
   - **Description:** Measures the validation error every $ N $ steps (or epochs) and reduces the learning rate by $ \lambda $ if the error has stopped dropping.
   - **Senior et al's Recommendation:** Advised for its adaptability to the model’s actual progress.


In [ ]:
# Performance based scheduling requires the use of a callback, so we can evaluate the performance of the network
# We can do it like this
lr0 = .01
model = build_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=lr0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)
history_performance_scheduling = model.fit(X_train, y_train, epochs=25,
                    validation_data=(X_valid, y_valid),
                    callbacks=[lr_scheduler])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_performance_scheduling.epoch, history_performance_scheduling.history["lr"], "bo-")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate", color='b')
plt.tick_params('y', colors='b')
plt.gca().set_xlim(0, 24)
plt.grid(True)

ax2 = plt.gca().twinx()
ax2.plot(history_performance_scheduling.epoch, history_performance_scheduling.history["val_loss"], "r^-")
ax2.set_ylabel('Validation Loss', color='r')
ax2.tick_params('y', colors='r')

plt.title("Reduce LR on Plateau", fontsize=14)
plt.show()


5. **1cycle Scheduling:**
   - **Description:** Briefly increases the learning rate halfway through training before decreasing it. Proposed by [Leslie N. Smith in 2018](https://arxiv.org/abs/1803.09820).
   - **Benefits (Smith, 2018):** Shown to lead to faster convergence and better performance. It allows the model to explore a wider range of parameters due to the higher learning rate in the middle of training.
   - **Formulation:** Starts with a low learning rate, increases it linearly for the first half of training, and then decreases it linearly for the second half, possibly followed by a few epochs with a very small learning rate to finalize training.


Implementing the 1-cycle learning rate scheduler in Keras is not hard, but it requires us to obtain an optimal learning rate first before creating a scheduler.  We did this above.  Here, I'm implementing the LR scheduler as a callback due to some of the complexities of working with tensors.

In [ ]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None,
                 last_iterations=None, last_lr=None):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr,
                                   self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)

In [ ]:
model = build_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])
n_epochs = 25
onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * n_epochs,
                             max_lr=0.1)
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid, y_valid),
                    callbacks=[onecycle])



Learning rate scheduling is a powerful tool in the training of neural networks. Different scheduling methods offer various advantages, and the choice of method can depend on the specific characteristics of the task, the dataset, and the desired training dynamics. While 1cycle scheduling has shown promising results for faster convergence, the recommendations of Senior et al. for performance or exponential scheduling highlight the importance of choosing a method that aligns with the model’s learning progress and the practitioner’s familiarity with the training dynamics.